In [52]:
import os
import pandas as pd
import numpy as np
import json
import glob
import sklearn
from sklearn.feature_selection import SelectKBest
from sklearn.feature_selection import chi2
from sklearn.feature_selection import f_classif
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import train_test_split
from sklearn.metrics import average_precision_score
from sklearn.tree import DecisionTreeClassifier
from sklearn import metrics
from sklearn.metrics import mean_squared_error
from sklearn.ensemble import GradientBoostingClassifier
from sklearn.ensemble import ExtraTreesClassifier
from sklearn.metrics import  classification_report
from sklearn.metrics import confusion_matrix
from imblearn.over_sampling import SMOTE
from imblearn.under_sampling import NearMiss

In [8]:
result=pd.read_csv("processed_data.csv")

In [9]:
result


,Unnamed: 0,windspeedKmph,winddirDegree,weatherCode,precipMM,visibility,pressure,cloudcover,DewPointF,WindGustKmph,...,DepDelayMinutes,OriginAirportID,DestAirportID,ArrTime,CRSArrTime,ArrDel15,ArrDelayMinutes,Origin,Dest,round_time
0,0,11,318,176,0.1,10,1023,100,47,17,...,17.0,10397,13204,101.0,51,0.0,10.0,ATL,MCO,0
1,1,18,315,122,0.0,10,1024,100,38,28,...,6.0,10397,13204,728.0,720,0.0,8.0,ATL,MCO,600
2,2,18,315,122,0.0,10,1024,100,38,28,...,37.0,10397,13930,753.0,725,1.0,28.0,ATL,ORD,600
3,3,18,317,122,0.0,10,1024,100,38,26,...,0.0,10397,13303,838.0,837,0.0,1.0,ATL,MIA,700
4,4,18,317,122,0.0,10,1024,100,38,26,...,0.0,10397,12892,922.0,915,0.0,7.0,ATL,LAX,700
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1851431,1851431,20,303,113,0.0,10,1014,0,58,22,...,9.0,14771,13930,513.0,519,0.0,0.0,SFO,ORD,2300
1851432,1851432,20,303,113,0.0,10,1014,0,58,22,...,5.0,14771,10397,615.0,632,0.0,0.0,SFO,ATL,2300
1851433,1851433,20,303,113,0.0,10,1014,0,58,22,...,0.0,14771,12478,735.0,750,0.0,0.0,SFO,JFK,2300
1851434,1851434,20,303,113,0.0,10,1014,0,58,22,...,0.0,14771,12478,655.0,704,0.0,0.0,SFO,JFK,2300


In [10]:
target=result["ArrDel15"]

In [11]:
features=result.copy()
features=features.drop(["ArrDel15","Unnamed: 0","date","airport","air","FlightDate","Origin","Dest"],axis=1)

In [12]:
features=features.drop(["ArrDelayMinutes","time","ArrTime","CRSArrTime","time","round_time"],axis=1)

In [13]:
selector=SelectKBest(f_classif,k=20).fit(features,target)

In [14]:
cols=selector.get_support(indices=True)

In [15]:
features_selt=features.iloc[:,cols]

In [16]:
features_selt

,windspeedKmph,winddirDegree,weatherCode,precipMM,visibility,pressure,cloudcover,DewPointF,WindGustKmph,tempF,WindChillF,int_time,Quarter,Month,DepTime,DepDel15,CRSDepTime,DepDelayMinutes,OriginAirportID,DestAirportID
0,11,318,176,0.1,10,1023,100,47,17,49,46,0,1,1,2347.0,1.0,2330,17.0,10397,13204
1,18,315,122,0.0,10,1024,100,38,28,40,32,600,1,1,606.0,0.0,600,6.0,10397,13204
2,18,315,122,0.0,10,1024,100,38,28,40,32,600,1,1,652.0,1.0,615,37.0,10397,13930
3,18,317,122,0.0,10,1024,100,38,26,41,33,700,1,1,642.0,0.0,645,0.0,10397,13303
4,18,317,122,0.0,10,1024,100,38,26,41,33,700,1,1,715.0,0.0,715,0.0,10397,12892
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1851431,20,303,113,0.0,10,1014,0,58,22,66,66,2300,3,9,2329.0,0.0,2320,9.0,14771,13930
1851432,20,303,113,0.0,10,1014,0,58,22,66,66,2300,3,9,2258.0,0.0,2253,5.0,14771,10397
1851433,20,303,113,0.0,10,1014,0,58,22,66,66,2300,3,9,2309.0,0.0,2310,0.0,14771,12478
1851434,20,303,113,0.0,10,1014,0,58,22,66,66,2300,3,9,2225.0,0.0,2230,0.0,14771,12478


In [17]:
#listing the selected features.
list(features_selt.columns)

[&#39;windspeedKmph&#39;,
 &#39;winddirDegree&#39;,
 &#39;weatherCode&#39;,
 &#39;precipMM&#39;,
 &#39;visibility&#39;,
 &#39;pressure&#39;,
 &#39;cloudcover&#39;,
 &#39;DewPointF&#39;,
 &#39;WindGustKmph&#39;,
 &#39;tempF&#39;,
 &#39;WindChillF&#39;,
 &#39;int_time&#39;,
 &#39;Quarter&#39;,
 &#39;Month&#39;,
 &#39;DepTime&#39;,
 &#39;DepDel15&#39;,
 &#39;CRSDepTime&#39;,
 &#39;DepDelayMinutes&#39;,
 &#39;OriginAirportID&#39;,
 &#39;DestAirportID&#39;]

In [18]:
train_feature,test_feature,train_target,test_target=train_test_split(features_selt,target,test_size=0.2,random_state=42)

In [19]:
#logistic regression classification
LogReg=LogisticRegression()
LogReg.fit(train_feature,train_target)
logit_predict=LogReg.predict(test_feature)

In [20]:
print(classification_report(test_target,logit_predict))

              precision    recall  f1-score   support

         0.0       0.92      0.98      0.95    292301
         1.0       0.89      0.68      0.77     77987

    accuracy                           0.92    370288
   macro avg       0.91      0.83      0.86    370288
weighted avg       0.91      0.92      0.91    370288



In [21]:
 #decision tree classifier
 dtc=DecisionTreeClassifier(random_state=0)
 dtc=dtc.fit(train_feature,train_target)
 tree_pred=dtc.predict(test_feature)
 8

In [22]:
print(classification_report(test_target,tree_pred))

              precision    recall  f1-score   support

         0.0       0.92      0.91      0.92    292301
         1.0       0.68      0.70      0.69     77987

    accuracy                           0.87    370288
   macro avg       0.80      0.81      0.80    370288
weighted avg       0.87      0.87      0.87    370288



In [23]:
#xgboost
xgb = GradientBoostingClassifier(random_state=0)
xgb.fit(train_feature,train_target)

GradientBoostingClassifier(random_state=0)

In [24]:
xgb_pred=xgb.predict(test_feature)

In [25]:
print(classification_report(test_target,xgb_pred))

              precision    recall  f1-score   support

         0.0       0.92      0.98      0.95    292301
         1.0       0.90      0.68      0.77     77987

    accuracy                           0.92    370288
   macro avg       0.91      0.83      0.86    370288
weighted avg       0.92      0.92      0.91    370288



In [26]:
#extra tree classifier
etc=ExtraTreesClassifier(n_estimators=100,random_state=0)
etc.fit(train_feature,train_target)
extratree_pred=etc.predict(test_feature)


In [27]:

print(classification_report(test_target,extratree_pred))

              precision    recall  f1-score   support

         0.0       0.93      0.96      0.94    292301
         1.0       0.83      0.73      0.77     77987

    accuracy                           0.91    370288
   macro avg       0.88      0.84      0.86    370288
weighted avg       0.91      0.91      0.91    370288



In [28]:
#oversampling method
#os = oversampling
os=SMOTE(random_state=2)
train_feature_os,train_target_os= os.fit_sample(train_feature,train_target)

In [29]:
#logistic reg
LogReg_os=LogisticRegression()
LogReg_os.fit(train_feature_os,train_target_os)
logit_predict_os=LogReg_os.predict(test_feature)

In [30]:
print(classification_report(test_target,logit_predict_os))

              precision    recall  f1-score   support

         0.0       0.94      0.93      0.93    292301
         1.0       0.74      0.78      0.76     77987

    accuracy                           0.90    370288
   macro avg       0.84      0.85      0.85    370288
weighted avg       0.90      0.90      0.90    370288



In [31]:
#decision tree classifier
dtc_os=DecisionTreeClassifier()
dtc_os.fit(train_feature_os,train_target_os)
tree_pred_os=dtc_os.predict(test_feature)

In [32]:
print(classification_report(test_target,tree_pred_os))

              precision    recall  f1-score   support

         0.0       0.92      0.90      0.91    292301
         1.0       0.66      0.70      0.68     77987

    accuracy                           0.86    370288
   macro avg       0.79      0.80      0.80    370288
weighted avg       0.87      0.86      0.86    370288



In [33]:
#xgboost
xgb_os = GradientBoostingClassifier(random_state=0)
xgb_os.fit(train_feature_os,train_target_os)
xgb_pred_os=xgb_os.predict(test_feature)

In [5]:
print(classification_report(test_target,xgb_pred_os))

NameError: name &#39;test_target&#39; is not defined

In [35]:
#extra tree classifier
etc_os=ExtraTreesClassifier(n_estimators=100,random_state=0)
etc_os.fit(train_feature_os,train_target_os)
extratree_pred_os=etc_os.predict(test_feature)


In [36]:
print(classification_report(test_target,extratree_pred_os))

              precision    recall  f1-score   support

         0.0       0.94      0.95      0.94    292301
         1.0       0.79      0.75      0.77     77987

    accuracy                           0.91    370288
   macro avg       0.86      0.85      0.86    370288
weighted avg       0.90      0.91      0.90    370288



In [23]:
#under-sampling
#us- under sampling
us =NearMiss()
train_feature_us,train_target_us=  us.fit_sample(train_feature,train_target)

In [38]:
#logistic reg
LogReg_us=LogisticRegression()
LogReg_us.fit(train_feature_us,train_target_us)
logit_predict_us=LogReg_us.predict(test_feature)

In [39]:
print(classification_report(test_target,logit_predict_us))


              precision    recall  f1-score   support

         0.0       0.94      0.84      0.89    292301
         1.0       0.57      0.80      0.67     77987

    accuracy                           0.83    370288
   macro avg       0.76      0.82      0.78    370288
weighted avg       0.86      0.83      0.84    370288



In [40]:
#decision tree classifier
dtc_us=DecisionTreeClassifier()
dtc_us.fit(train_feature_us,train_target_us)
tree_pred_us=dtc_us.predict(test_feature)

In [41]:
print(classification_report(test_target,tree_pred_us))

              precision    recall  f1-score   support

         0.0       0.93      0.61      0.74    292301
         1.0       0.36      0.83      0.51     77987

    accuracy                           0.66    370288
   macro avg       0.65      0.72      0.62    370288
weighted avg       0.81      0.66      0.69    370288



In [42]:
#xgboost
xgb_us = GradientBoostingClassifier(random_state=0)
xgb_us.fit(train_feature_us,train_target_us)
xgb_pred_us=xgb_us.predict(test_feature)

In [43]:
print(classification_report(test_target,xgb_pred_us))

              precision    recall  f1-score   support

         0.0       0.94      0.78      0.85    292301
         1.0       0.50      0.82      0.62     77987

    accuracy                           0.79    370288
   macro avg       0.72      0.80      0.73    370288
weighted avg       0.85      0.79      0.80    370288



In [24]:
#extra tree classifier
etc_us=ExtraTreesClassifier(n_estimators=100,random_state=0)
etc_us.fit(train_feature_us,train_target_us)
extratree_pred_us=etc_us.predict(test_feature)

In [25]:
print(classification_report(test_target,extratree_pred_us))

              precision    recall  f1-score   support

         0.0       0.95      0.69      0.80    292301
         1.0       0.42      0.85      0.56     77987

    accuracy                           0.72    370288
   macro avg       0.68      0.77      0.68    370288
weighted avg       0.84      0.72      0.75    370288



In [40]:
Arr_del_index=list(test_feature.index)


In [48]:
test_feature["ArrDelayMinutes"]=result["ArrDelayMinutes"][Arr_del_index]

In [49]:
test_feature

,windspeedKmph,winddirDegree,weatherCode,precipMM,visibility,pressure,cloudcover,DewPointF,WindGustKmph,tempF,...,int_time,Quarter,Month,DepTime,DepDel15,CRSDepTime,DepDelayMinutes,OriginAirportID,DestAirportID,ArrDelayMinutes
1778209,11,55,113,0.0,10,1013,0,29,16,64,...,1000,4,10,923.0,0.0,930,0.0,14771,12892,26.0
737512,22,242,113,0.0,10,1016,1,32,37,39,...,700,1,1,650.0,0.0,700,0.0,12478,13303,0.0
36012,18,152,122,0.0,10,1014,100,61,24,67,...,1100,1,3,1125.0,1.0,1050,35.0,10397,12892,33.0
1079740,25,295,116,0.0,10,1009,33,55,27,69,...,1200,2,4,1207.0,1.0,1130,37.0,12892,14747,39.0
1093872,15,229,113,0.0,10,1012,0,58,18,72,...,1400,2,6,1402.0,0.0,1400,2.0,12892,14771,3.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1792665,4,20,113,0.0,10,1022,0,51,9,63,...,2200,4,12,2130.0,0.0,2135,0.0,14771,13303,0.0
1044057,18,251,116,0.0,10,1017,59,59,19,63,...,1600,4,11,1625.0,0.0,1625,0.0,12892,11618,0.0
319158,11,99,116,0.0,10,1019,10,25,24,52,...,1700,4,11,1810.0,1.0,1650,80.0,11292,14771,57.0
1545862,8,85,113,0.0,10,1017,0,22,10,74,...,1000,2,4,934.0,0.0,940,0.0,14107,14747,0.0


In [72]:
test_feature["pred"]=extratree_pred_us

In [74]:
test_feature

,windspeedKmph,winddirDegree,weatherCode,precipMM,visibility,pressure,cloudcover,DewPointF,WindGustKmph,tempF,...,Quarter,Month,DepTime,DepDel15,CRSDepTime,DepDelayMinutes,OriginAirportID,DestAirportID,ArrDelayMinutes,pred
1778209,11,55,113,0.0,10,1013,0,29,16,64,...,4,10,923.0,0.0,930,0.0,14771,12892,26.0,0.0
737512,22,242,113,0.0,10,1016,1,32,37,39,...,1,1,650.0,0.0,700,0.0,12478,13303,0.0,1.0
36012,18,152,122,0.0,10,1014,100,61,24,67,...,1,3,1125.0,1.0,1050,35.0,10397,12892,33.0,1.0
1079740,25,295,116,0.0,10,1009,33,55,27,69,...,2,4,1207.0,1.0,1130,37.0,12892,14747,39.0,1.0
1093872,15,229,113,0.0,10,1012,0,58,18,72,...,2,6,1402.0,0.0,1400,2.0,12892,14771,3.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1792665,4,20,113,0.0,10,1022,0,51,9,63,...,4,12,2130.0,0.0,2135,0.0,14771,13303,0.0,1.0
1044057,18,251,116,0.0,10,1017,59,59,19,63,...,4,11,1625.0,0.0,1625,0.0,12892,11618,0.0,0.0
319158,11,99,116,0.0,10,1019,10,25,24,52,...,4,11,1810.0,1.0,1650,80.0,11292,14771,57.0,1.0
1545862,8,85,113,0.0,10,1017,0,22,10,74,...,2,4,934.0,0.0,940,0.0,14107,14747,0.0,0.0


In [75]:
test_feature=test_feature[test_feature["pred"]==1.0]

In [76]:
test_feature

,windspeedKmph,winddirDegree,weatherCode,precipMM,visibility,pressure,cloudcover,DewPointF,WindGustKmph,tempF,...,Quarter,Month,DepTime,DepDel15,CRSDepTime,DepDelayMinutes,OriginAirportID,DestAirportID,ArrDelayMinutes,pred
737512,22,242,113,0.0,10,1016,1,32,37,39,...,1,1,650.0,0.0,700,0.0,12478,13303,0.0,1.0
36012,18,152,122,0.0,10,1014,100,61,24,67,...,1,3,1125.0,1.0,1050,35.0,10397,12892,33.0,1.0
1079740,25,295,116,0.0,10,1009,33,55,27,69,...,2,4,1207.0,1.0,1130,37.0,12892,14747,39.0,1.0
1423204,12,81,116,0.0,10,1014,47,49,15,61,...,2,5,1349.0,0.0,1350,0.0,13930,14747,21.0,1.0
182611,6,222,356,2.2,8,1016,38,71,10,80,...,3,7,809.0,0.0,810,0.0,11057,10397,5.0,1.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
937989,6,166,113,0.0,10,1015,0,45,6,77,...,4,11,1748.0,1.0,1725,23.0,12892,11298,15.0,1.0
1535529,5,96,113,0.0,10,1021,0,34,7,50,...,1,2,1028.0,1.0,1000,28.0,14107,14771,25.0,1.0
1307026,9,225,143,0.0,6,1006,100,35,12,39,...,1,1,1711.0,0.0,1715,0.0,13930,11292,0.0,1.0
1792665,4,20,113,0.0,10,1022,0,51,9,63,...,4,12,2130.0,0.0,2135,0.0,14771,13303,0.0,1.0


In [77]:
test_feature=test_feature.drop(["pred"],axis=1)

In [81]:
test_feature.to_csv("pipeline_data.csv")